In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import tensorflow
import cv2
from keras.models import load_model
import os

Using TensorFlow backend.


In [16]:
from tkinter import *

def show_entry_fields():
    print("First Name: %s\nLast Name: %s" % (e1.get(), e2.get()))
    master.quit

master = Tk()
Label(master, text="First Name").grid(row=0)
Label(master, text="Last Name").grid(row=1)

e1 = Entry(master)
e2 = Entry(master)

e1.grid(row=0, column=1)
e2.grid(row=1, column=1)

Button(master, text='Quit', command=master.quit).grid(row=3, column=0, sticky=W, pady=4)
Button(master, text='Show', command=show_entry_fields).grid(row=3, column=1, sticky=W, pady=4)

mainloop( )

First Name: shivang
Last Name: chopra


In [2]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')
FRmodel = load_model('./face-rec_Google.h5')

/usr/local/lib/python3.6/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
print(FRmodel.count_params())

3743280


In [4]:
card_data = pd.read_csv('./Card Data.csv')
card_data.head()

,Card Type,Card Number,CVV,Expires
0,Visa,4556035157638848,397,04/2022
1,MasterCard,4716487626397851,395,12/2022
2,Visa,4916353991301746,316,03/2023
3,Visa,4929299429259144,780,05/2023
4,MasterCard,5276921737271851,145,04/2021


In [5]:
def img_to_embedding(image, model):
    image = cv2.resize(image, (96, 96)) 
    img = image[...,::-1]
    img = np.around(np.transpose(img, (2,0,1))/255.0, decimals=12)
    x_train = np.array([img])
    embedding = model.predict(x_train)
    return embedding

In [51]:
def distance(x1,x2):
    print(x2,x1)
    return np.sqrt(((x2-x1)**2).sum())

In [52]:
def embedding_present(emb,thresh,cur_card):
    # TO DO
    ctr=0
    face_data = []
    card_data = []
    for ix in os.listdir('./data_today/'):
        if ix.endswith('.npy'):
            data_item = np.load('./data_today/' + str(ix))
            card_no = data_item[-1]
            data_item = data_item[:-1]
            print(data_item)
            face_data.append(data_item)
            card_data.append(card_no)
            ctr += 1
    idx = 0
    for ix in face_data:
        print(distance(ix,emb))
        print(card_data[idx])
        if distance(ix,emb)<thresh and cur_card!=card_data[idx]:
            return True,ctr
        idx+=1
    return False,ctr

In [8]:
r = tk.Tk() 
r.title('Counting Seconds') 
button = tk.Button(r, text='Stop', width=25, command=r.destroy) 
button.pack()
r.mainloop()

In [58]:
card_no = input("Enter last 4 digits of card:")
cvv = input("Enter CVV:")
card_no = float(card_no)
cvv = float(cvv)
start = False
temp = False
cap = cv2.VideoCapture(0)
count=1
while True:
    ret, frame = cap.read()
    if ret==False:
        continue
    frame = cv2.flip(frame, 1)
    cv2.imshow("Faces", frame)
    key = cv2.waitKey(1)
    if key==113:
        break
    if key==115:
        start = True
    if start == False:
        continue
    if temp == True:
        break
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    k=1
    faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
    for face in faces[:1]:
        x,y,w,h = face
        offset = 7
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(100,100))
        emb = img_to_embedding(face_section,FRmodel)
        emb=np.reshape(emb,(128,))
        print(type(emb[0]))
        pre, ctr = embedding_present(emb,0.1,card_no)
        if(pre):
            print("ALARM")
            temp = True
#             cv2.putText(frame, "ALARM",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2,cv2.LINE_AA)
        else:
            # Save Embedding
            print("SAVING")
            temp = True
#             cv2.putText(frame, "SAVING DATA",(x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2,cv2.LINE_AA)
            emb = np.append(emb,card_no)
            print(emb.shape)
            np.save('./data_today/' + str(ctr), emb)
            ctr+=1
cap.release()
cv2.destroyAllWindows()

Enter last 4 digits of card:2341
Enter CVV:123
<class 'numpy.float32'>
[ 0.10055459 -0.0144635  -0.03411391 -0.04046786 -0.00578374  0.21636379
 -0.13875291  0.01776691 -0.024172   -0.07282005  0.09245242  0.00944639
  0.00583351  0.05776818  0.10285748  0.14325395 -0.01602217  0.03094343
 -0.00277294 -0.12154169 -0.04873688  0.03470003 -0.05647378 -0.14257477
  0.01757035 -0.01331341  0.11812568 -0.0928046   0.02551125  0.06005829
  0.15924466  0.03780581  0.11242409  0.05231652  0.05930289 -0.11817392
  0.03130753  0.11544863  0.04489526  0.00211491  0.01559704  0.08318797
  0.0298853  -0.0123855   0.01487324  0.14608283  0.0250263  -0.12570013
 -0.05976643  0.06035124 -0.14763388  0.16190434 -0.07276715  0.18829376
 -0.09481242 -0.07484999 -0.04290181  0.0170518   0.079011    0.10622992
  0.01777478 -0.01313861  0.08887708  0.01848042  0.05613836 -0.02718945
 -0.0293042  -0.00169029  0.06558694  0.09857052 -0.11338758  0.03739107
 -0.01173426 -0.00023317  0.06327636  0.04785518  0.0